<a href="https://colab.research.google.com/github/YixinFan11/Big-Data-Tweeter-Sentiment-Analysis/blob/master/Big_Data_Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading the data from Kaggle

In [ ]:
## Prerequisites
#!pip install kaggle --upgrade
#!cp kaggle.json ~/.kaggle/
#!chmod 600 ~/.kaggle/kaggle.json

! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json



mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
## Downloading data command
!kaggle datasets download -d manchunhui/us-election-2020-tweets

us-election-2020-tweets.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!mkdir content/data
!unzip us-election-2020-tweets.zip -d data

mkdir: cannot create directory ‘content/data’: No such file or directory
Archive:  us-election-2020-tweets.zip
replace data/hashtag_donaldtrump.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/hashtag_joebiden.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


## Download Pyspark

In [ ]:
!pip install pyspark 

## Import Header Files

In [ ]:
## Python 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import re


# stopwords, tokenizer, stemmer
import nltk  
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.probability import FreqDist

# spell correction, lemmatization
from textblob import TextBlob
from textblob import Word

## Spark
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType , FloatType
from pyspark.sql.functions import lit
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import udf

# Create Spark Session
spark = SparkSession.builder.master("local[1]").appName('Twitter Sentiment Analysis').getOrCreate()

## Reading the Data

#### Defining the Schema

In [ ]:

# Created the schema that we need 
# # Fucking Brute Force I dont like it
mySchema = StructType([ StructField('created_at', StringType(), True)\
                       ,StructField('tweet_id', FloatType(), True)\
                       ,StructField('tweet',StringType(),True)\
                       ,StructField('likes',FloatType(),True)\
                       ,StructField('retweet_count',FloatType(),True)\
                       ,StructField('source',StringType(),True)\
                       ,StructField('user_id',FloatType(),True)\
                       ,StructField('user_name',StringType(),True)\
                       ,StructField('user_screen_name',StringType(),True)\
                       ,StructField('user_description',StringType(),True)\
                       ,StructField('user_join_date',StringType(),True)\
                       ,StructField('user_followers_count',FloatType(),True)\
                       ,StructField('user_location',StringType(),True)\
                       ,StructField('lat',FloatType(),True)\
                       ,StructField('long',FloatType(),True)\
                       ,StructField('city',StringType(),True)\
                       ,StructField('country',StringType(),True)\
                       ,StructField('continent',StringType(),True)\
                       ,StructField('state',StringType(),True)\
                       ,StructField('state_code',StringType(),True)\
                       ,StructField('collected_at',StringType(),True)\
                       ])

In [ ]:
# Creating spark data frame
# This shit was so hard to do it was not reading multiline data at first it killed me
# Took me 6 hours to figure this shit 
# Now have to change the schema to different data type

trump_df = spark.read.option('multiline',True)\
                     .option('header', 'true')\
                     .option('lineterminator',"\n")\
                     .option('inferSchema', "true")\
                     .schema(mySchema)\
                     .csv('/content/data/hashtag_donaldtrump.csv')
trump_df.cache()
print('Trump DataFrame Schema')
trump_df.printSchema()


biden_df = spark.read.option('multiline',True)\
                     .option('header', 'true')\
                     .option('lineterminator',"\n")\
                     .option('inferSchema', "true")\
                     .schema(mySchema)\
                     .csv('/content/data/hashtag_joebiden.csv')

biden_df.cache()
print('Biden DataFrame Schema')
biden_df.printSchema()

#### Trump Data Stats

In [ ]:
# Took me 35 seconds to run
trump_df.show(5)

+-------------------+-------------+--------------------+-----+-------------+------------------+------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+----------+--------------------+-------------+--------------------+----------+--------------------+
|         created_at|     tweet_id|               tweet|likes|retweet_count|            source|     user_id|           user_name|user_screen_name|    user_description|     user_join_date|user_followers_count|       user_location|      lat|       long|      city|             country|    continent|               state|state_code|        collected_at|
+-------------------+-------------+--------------------+-----+-------------+------------------+------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+----------+--------------------+-------------+-

In [ ]:
# Took me 2 minutes to run one command carefull while running
trump_df.summary().show()

#### Biden Data Stats

In [ ]:
biden_df.show(5)

In [ ]:
# Took me 1 minute 21 seconds to run this command carefull while running
biden_df.summary().show()

#### Concatinating Data 

In [ ]:
## Adding a column to the dataframe 
trump_df = trump_df.withColumn("Candidate", lit('TRUMP'))
trump_df.show()

In [ ]:
biden_df = biden_df.withColumn("Candidate", lit('BIDEN'))
biden_df.show()

In [ ]:
data = trump_df.union(biden_df)
data.cache()
data.show()

## EDA



```
someone please plot graphs for each one of them
```

In [ ]:
## Checking the data imbalance
data.groupBy('Candidate').count().withColumnRenamed("count","Number of Data").show()
## The dataset is imbalance but not serious(Yixin)

In [ ]:
## Comparrsion for the number of tweets
data.groupby('Candidate').agg(F.expr('count(tweet)')\
                              .alias('Number of Tweets')).show()
## After delete the null value in tweets, the imbalance status does not change too much(Yixin)

In [ ]:
# comparison of likes
data.groupby('Candidate').agg(F.expr('count(likes)')\
                              .alias('Number of Likes')).show()
### Better to add up than just count rows(Yixin)

In [ ]:
# Can someone please remove the null that is being shown also show only top 10 rows 
data.na.drop(subset=["Country"]).show()
data.groupby('Country').count()\
.withColumnRenamed('count','Number of Data')\
.sort(F.desc('Number of Data')).show()
## dont think this will make any sense, since most of countries are null value(Yixin)

In [ ]:
x=316181+59192+55419+38786+32820+32245+26411+18524+14201+10284+9829+9147+8352+8287+6556+5259+5136+4229+4041
print(x)
x1 =data.select('Created_at').count()
print(x1)

In [ ]:
# Making USA to US 
data = data.withColumn('Country', regexp_replace('Country', 'United States of America', 'US'))
data = data.withColumn('Country', regexp_replace('Country', 'United States', 'US'))

## Make sure by checking the count of the data 
data.groupby('Country').count()\
.withColumnRenamed('count','Number of Data')\
.sort(F.desc('Number of Data')).show()

## dont think this will make any sense, since most of countries are null value(Yixin)

## Preprocessing the Data(Tweets)

In [ ]:
!pip install nltk
nltk.download("stopwords")
nltk.download('punkt')

In [ ]:
##### TRY this one by your self
# Refer to user defined functions for this application
text = u' This dog \U0001f602 <a href="https://stackoverflow.com/questions/49840965/how-to-add-hyperlinks-to-strings-in-python">TIC123</a> '
print(text) 

def preprocess_tweets(text):#, stem=False, lemmatize=False):
    # Make everything in lower case
    text = text.lower()
    # Remove links, special characters , numbers punctuation etc. 
    # Refer - https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python 
    # emoji_pattern = re.compile("["
    #     u"\U0001F600-\U0001F64F"  # emoticons
    #     u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    #     u"\U0001F680-\U0001F6FF"  # transport & map symbols
    #     u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    #                        "]+", flags=re.UNICODE)
    # Bottom one is also working try it with the example that I am using 
    # I will comment this out so you guys can try
    chars_and_links= r'\d+|http?\S+|[^A-Za-z0-9]+'
    text = re.sub(chars_and_links,' ',text)
    #text = re.sub(emoji_pattern,' ',text)
    return text

print(preprocess_tweets(text))

In [ ]:
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

def preprocess_tweets(text, stem=False, lemmatize=False):
    ## If statement for fault tolerance
    if isinstance(text,str):
        text = text.lower()
        # Remove links, special characters , numbers punctuation etc. 
        chars_and_links= r'\d+|http?\S+|[^A-Za-z0-9]+'
        # Cleaned the text
        text = re.sub(chars_and_links,' ',text)

        filtered_text = []
        # Tokenize the text
        words = word_tokenize(text) 
        # Remove stopwords and stem
        for word in words:
            # Words in stop words or not
            if not word in stop_words:
                if stem:
                    filtered_text.append(ps.stem(word))
                elif lemmatize:
                    filtered_text.append(Word(word).lemmatize())
                else:
                    filtered_text.append(word)
        # Return the filtered text
        return filtered_text
    else:
        return text

In [ ]:
# Drop all the NAN values
#data.na.drop(subset=["tweet"]).show()


## Now to apply this to the text we need to create a user defined function
## Refer https://sparkbyexamples.com/pyspark/pyspark-udf-user-defined-function/

## Created my udf
Preprocess_UDF = udf(lambda x: preprocess_tweets(x))  
# "marketplace","star_rating").distinct().show(5)
# data = data.select(Preprocess_UDF(F.col("tweet")).alias("tweet")).show(5)
#.select([F.col(c).cast('float').alias(c) for c in df.columns])

# data = data.select(Preprocess_UDF(F.col('tweet').alias('Tokenized_tweet'))).collect()
# data.show()

In [ ]:
#data_res.show(5)

In [ ]:
#data.show(5)

In [ ]:
def sentiment_analysis(df):
    
    # Determine polarity and subjectivity
    data['Polarity'] = data['tweetNew'].apply(lambda x: TextBlob(' '.join(x)).sentiment.polarity)
    df['Subjectivity'] = df['tweetNew'].apply(lambda x: TextBlob(' '.join(x)).sentiment.subjectivity)
    
    # Classify overall sentiment
    df.loc[df.Polarity > 0,'Sentiment'] = 'positive'
    df.loc[df.Polarity == 0,'Sentiment'] = 'neutral'
    df.loc[df.Polarity < 0,'Sentiment'] = 'negative'
    
    return df[['tweet','tweetNew','Polarity','Subjectivity','Sentiment']].head()

